# SVM

## Setup

In [51]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

為了防止一些Sklearn更新的不必要警告:

In [16]:
# import warnings filter
from warnings import simplefilter
#ugnore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [17]:
iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values#只用兩個變數
y = (iris.target == 2)  # 只預測花種類是否是virginica

## Linear SVM Classification

用最簡單的線性SVM分類iris資料集

In [18]:
from sklearn.svm import SVC

linear_kernel_svm_clf = make_pipeline(StandardScaler(),
                                    SVC(kernel="linear", C=1))
linear_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, kernel='linear'))])

自己創建兩個新的observation預測看看

In [23]:
X_new = [[5.5, 1.7], [5.0, 1.5]]
linear_kernel_svm_clf.predict(X_new)

array([ True, False])

第一種植物被歸類為 Iris virginica，而第二種則不是。 讓我們看看 SVM 用來做出這些預測的分數。 這些測量每個observation和決策邊界之間的距離。

In [22]:
linear_kernel_svm_clf.decision_function(X_new)

array([ 0.83293241, -0.40504826])

# Polynomial Kernel

找到正確的超參數值的一種常見方法是使用網格搜索。 首先進行非常粗略的網格搜索，然後圍繞找到的最佳值進行更精細的網格搜索通常會更快。 充分了解每個超參數的實際作用還可以幫助您在超參數空間的正確部分進行搜索。

In [26]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
    ('scale', StandardScaler()),
    ('svc', SVC(kernel="poly")),
])

param_grid = [
    {'svc__C': [0.1,1, 10, 100], 
     'svc__gamma':[1,0.1,0.01,0.001],
     'svc__degree': [2, 3, 4, 5, 6]}
]

grid_search_SVM = GridSearchCV(poly_kernel_svm_clf, param_grid, cv = StratifiedKFold(n_splits=5),  scoring= 'accuracy')

grid_search_SVM.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('svc', SVC(kernel='poly'))]),
             param_grid=[{'svc__C': [0.1, 1, 10, 100],
                          'svc__degree': [2, 3, 4, 5, 6],
                          'svc__gamma': [1, 0.1, 0.01, 0.001]}],
             scoring='accuracy')

In [29]:
# extra code – displays the random search results
cv_res = pd.DataFrame(grid_search_SVM.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res = cv_res[["param_svc__C",
                 "param_svc__gamma", "param_svc__degree", "split0_test_score",
                 "split1_test_score", "split2_test_score", "split3_test_score", "split4_test_score", "mean_test_score"]]
score_cols = ["split0_cv", "split1_cv", "split2_cv","split3_cv","split4_cv", "mean_test_accuracy"]
cv_res.columns = ["C", "gamma", "degree"] + score_cols
cv_res[score_cols] = cv_res[score_cols].round().astype(np.int64)
cv_res.head()

,C,gamma,degree,split0_cv,split1_cv,split2_cv,split3_cv,split4_cv,mean_test_accuracy
72,100,1,5,1,1,1,1,1,1
64,100,1,3,1,1,1,1,1,1
52,10,1,5,1,1,1,1,1,1
24,1,1,3,1,1,1,1,1,1
44,10,1,3,1,1,1,1,1,1


In [30]:
grid_search_SVM.best_estimator_

Pipeline(steps=[('scale', StandardScaler()),
                ('svc', SVC(C=100, gamma=1, kernel='poly'))])

In [32]:
X_new = [[5.5, 1.7], [5.0, 1.5]]
grid_search_SVM.best_estimator_.predict(X_new)

array([ True, False])

### Gaussian RBF Kernel

In [41]:
rbf_kernel_svm_clf = Pipeline([
    ('scale', StandardScaler()),
    ('svc', SVC(kernel="rbf")),
])

param_grid = [
    {'svc__C': [0.001, 0.01, 0.1,1, 10, 20, 50, 100], 
     'svc__gamma':[100,50, 20, 10,5,1,0.1,0.01,0.001]}
]

grid_search_SVM = GridSearchCV(rbf_kernel_svm_clf, param_grid, cv = StratifiedKFold(n_splits=5),  scoring= 'accuracy')

grid_search_SVM.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid=[{'svc__C': [0.001, 0.01, 0.1, 1, 10, 20, 50, 100],
                          'svc__gamma': [100, 50, 20, 10, 5, 1, 0.1, 0.01,
                                         0.001]}],
             scoring='accuracy')

In [42]:
# extra code – displays the random search results
cv_res = pd.DataFrame(grid_search_SVM.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res = cv_res[["param_svc__C",
                 "param_svc__gamma", "split0_test_score",
                 "split1_test_score", "split2_test_score", "split3_test_score", "split4_test_score", "mean_test_score"]]
score_cols = ["split0_cv", "split1_cv", "split2_cv","split3_cv","split4_cv", "mean_test_accuracy"]
cv_res.columns = ["C", "gamma"] + score_cols
cv_res[score_cols] = cv_res[score_cols].round().astype(np.int64)
cv_res.head()

,C,gamma,split0_cv,split1_cv,split2_cv,split3_cv,split4_cv,mean_test_accuracy
38,10,20,1,1,1,1,1,1
48,20,10,1,1,1,1,1,1
56,50,20,1,1,1,1,1,1
47,20,20,1,1,1,1,1,1
65,100,20,1,1,1,1,1,1


In [43]:
grid_search_SVM.best_estimator_

Pipeline(steps=[('scale', StandardScaler()), ('svc', SVC(C=10, gamma=20))])

有這麼多內核可供選擇，您如何決定使用哪一個？根據經驗，您應該始終首先嘗試線性內核（請記住，LinearSVC 比 SVC(kernel="linear") 快得多），尤其是在訓練集非常大或具有大量特徵的情況下。 如果訓練集不是太大，也可以試試Gaussian RBF kernel； 在大多數情況下效果很好。 然後，如果您有空閒時間和計算能力，您還可以使用交叉驗證和網格搜索試驗其他幾個內核，特別是如果有專門針對您的訓練集數據結構的內核。

* LinearSVC 類基於 liblinear 庫，它實現了線性 SVM 的優化算法。 它不支持kernal trick，但它與訓練observation的數量和特徵的數量成線性關係：其訓練時間大致為 O(m × n)。 如果需要非常高的精度，則該算法需要更長的時間。 這是由容差超參數 ε（在 Scikit-Learn 中稱為 tol）控制的。 在大多數分類任務中，默認值的ε就可以了。


* SVC 類基於 libsvm 庫，它支持kernal trick的算法。 訓練時間複雜度通常在O(m^2×n)和O(m^3×n)之間。 不幸的是，這意味著當訓練observation的數量變大（例如，數十萬個observation）時，它會變得非常慢。 該算法非常適合複雜但中小型訓練集。 它可以很好地擴展特徵的數量，尤其是稀疏特徵（當每個observation大部分特徵都是零）。 在這種情況下，該算法訓練時間大致與每個observation的非零特徵的平均數量成比例。


## SVM Regression(SVR)


SVM 回歸不是試圖在兩個類之間擬合最大可能的街道同時限制margin violations，而是嘗試在街道上擬合盡可能多的實例同時限制margin violations（即街道上的observation). 街道的寬度由超參數 ε 控制。

### Linear SVR

In [ ]:
from sklearn.svm import LinearSVR

In [45]:
# extra code – these 3 lines generate a simple linear dataset
np.random.seed(42)
X = 2 * np.random.rand(50, 1)
y = 4 + 3 * X[:, 0] + np.random.randn(50)

In [46]:
svm_reg = make_pipeline(StandardScaler(),
                        LinearSVR(epsilon=0.5, random_state=42))
svm_reg.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvr', LinearSVR(epsilon=0.5, random_state=42))])

### Polynomial SVR(非線性)

In [49]:
from sklearn.svm import SVR

In [47]:
# extra code – these 3 lines generate a simple quadratic dataset
np.random.seed(42)
X = 2 * np.random.rand(50, 1) - 1
y = 0.2 + 0.1 * X[:, 0] + 0.5 * X[:, 0] ** 2 + np.random.randn(50) / 10

In [50]:
svm_poly_reg = make_pipeline(StandardScaler(),
                             SVR(kernel="poly", degree=2, C=0.01, epsilon=0.1))
svm_poly_reg.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(C=0.01, degree=2, kernel='poly'))])